In [1]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import keyboard # to interrupt while loops

# for math and calculating time and shit
import time
import math
import random


# To import a behavior agents for controlling car
from agents.navigation.basic_agent import BasicAgent
from agents.navigation.behavior_agent import BehaviorAgent

In [2]:
# connect to the sim, create the world
client = carla.Client('localhost', 2000)

# the town we've been using is town04, feel free to load different towns, goes to 012 i believe
client.load_world('Town04')

#define environment/world and get possible places to spawn a car
world = client.get_world()


spawn_points = world.get_map().get_spawn_points()

# function for clearing da world
def clear_world():
    # clears all vehicles + sensors and sets them to None
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
        actor = None
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
        sensor = None
        

In [53]:
# vehicle setup
class Vehicle:
    def __init__(self, world, vehicle_id, collision_callback, **callback_kwargs):
        
        # intialize reference world + diff spawn points + list of diff waypoints
        self.world = world
        self.spawn_points = world.get_map().get_spawn_points()
        self.waypoint_list = world.get_map().generate_waypoints(distance=2.0)
        self.current_waypoint = None # current destination of car
        
        # vehicle variables
        self.vehicle_id = vehicle_id
        self.vehicle = None
        self.agent = None
        
        self.collision_sensor = None
        self.collision_callback = collision_callback
        self.callback_kwargs = callback_kwargs # arguments for callback to customize respawn characteristics
        
        # picture capturing variables
        self.camera_sem = None
        self.camera_rgb = None
        self.camera_data = None
        
        
        # state constants for intervals between pictures
        self.interval = 7.5 # set interval to 7.5 seconds
        self.sharp_steer_interval = 0.1
        self.light_steer_interval = 0.75
        self.last_capture_time = time.time() # track last time pic was taken
        
        
    
    def spawn(self, blueprint=None, spawn_point=0):
        print("spawning vehicle: ", self.vehicle_id)
        # default blueprint is model 3
        if blueprint is None:
            blueprint = self.world.get_blueprint_library().filter('*model3*')[0] # it's an array so just return the first model
        
        # spawn the vehicle
        start_point = spawn_points[spawn_point]
        self.vehicle = self.world.try_spawn_actor(blueprint, start_point)
        
        # move simulator view to the car
        spectator = world.get_spectator()
        start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
        spectator.set_transform(start_point)
        
        if self.vehicle is None:
            raise Exception("Failed to spawn vehicle")
        
        # intialize steering
        control = carla.VehicleControl(throttle=0.0, steer=0.0, brake=0.0)
        self.vehicle.apply_control(control)
        #lights always on
        self.vehicle.set_light_state(carla.VehicleLightState.LowBeam)

        # setup the rest of the vehicle
        self.setup_collision_sensor()
        self.setup_cameras() 
        self.setup_agent()
        
    def setup_collision_sensor(self):
        # get collision sensor blueprint
        collision_bp = self.world.get_blueprint_library().find('sensor.other.collision')

        # Set the transform relative to the vehicle
        transform = carla.Transform(carla.Location(x=1.5, z=2.4))  # Front of the vehicle

        # Spawn the collision sensor and attach it to the vehicle
        self.collision_sensor = self.world.spawn_actor(collision_bp, transform, attach_to=self.vehicle)

        # Listen for collision events
        self.collision_sensor.listen(lambda event: self.collision_callback(event, self.vehicle, **self.callback_kwargs))  
        
    def setup_cameras(self):
        blueprint_library = self.world.get_blueprint_library()

        # Semantic camera setup
        camera_bp = blueprint_library.find('sensor.camera.semantic_segmentation')
        camera_bp.set_attribute('image_size_x', '640')
        camera_bp.set_attribute('image_size_y', '480')
        camera_bp.set_attribute('fov', '90')
        
        self.camera_sem = world.spawn_actor(camera_bp,carla.Transform(carla.Location(z=1.3, x=1.4)), attach_to=self.vehicle)

        # RGB camera setup
        camera_bp = blueprint_library.find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', '640')
        camera_bp.set_attribute('image_size_y', '480')
        camera_bp.set_attribute('fov', '90')
        
        self.camera_rgb = world.spawn_actor(camera_bp, carla.Transform(carla.Location(z=1.3, x=1.4)), attach_to=self.vehicle)

        # Callbacks for camera data
        def sem_callback(image, data_dict):
            image.convert(carla.ColorConverter.CityScapesPalette)
            data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

        def rgb_callback(image, data_dict):
            data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

        self.camera_data = {'sem_image': np.zeros((480, 640, 4)), 'rgb_image': np.zeros((480, 640, 4))}

        self.camera_sem.listen(lambda image: sem_callback(image, self.camera_data))
        self.camera_rgb.listen(lambda image: rgb_callback(image, self.camera_data))

    def setup_agent(self):

#         # start a basic agent for the car
#         self.agent = BasicAgent(self.vehicle)

        # give it a normal profile
        self.agent = BehaviorAgent(self.vehicle, behavior='normal') # maybe set to aggressive?

        # usually will just be overwritten just a placeholder location (maybe put self.waypoint here idk)
        self.current_waypoint = spawn_points[2].location # for now we just take the second spawn point
                                      # later use a CARLA location at self.x + 10 or something
            
        currLocation = self.vehicle.get_transform().location
        distance = self.distance(self.current_waypoint, currLocation)
        print(distance)
              
        self.agent.set_destination(self.current_waypoint)
        
    def destroy(self):
        
        print("destroying vehicle: ", self.vehicle_id)
        if self.collision_sensor is not None and self.collision_sensor.is_alive:
            self.collision_sensor.stop() # idk if this function exists
            self.collision_sensor.destroy()
            self.collision_sensor = None
            
        if self.vehicle is not None and self.vehicle.is_alive:
            self.vehicle.destroy()
            self.vehicle = None
        
        if self.camera_sem is not None and self.camera_sem.is_alive:
            self.camera_sem.destroy()
            self.camera_sem = None
            
        if self.camera_rgb is not None and self.camera_rgb.is_alive:
            self.camera_rgb.destroy()
            self.camera_rgb = None
        
        if self.agent:
            self.agent = None
            
            
            
    # helper functions for running:
    
    
    def print_vehicle_controls(self):
        control = self.vehicle.get_control()
        normalized_steer = (control.steer + 1) /2
        normalized_throttle = (control.throttle)
        normalized_brake = (control.brake)
        print("Steer: {:.3f}, Throttle: {:.3f}, Brake: {:.3f}".format(normalized_steer, normalized_throttle, normalized_brake))

    def grab_and_display_image(self):
        #     print_vehicle_controls(vehicle)

        rgb_im = self.camera_data['rgb_image']
        sem_im = self.camera_data['sem_image']

#         # display images
#         im_h = cv2.hconcat([rgb_im,sem_im])
#         cv2.imshow('2 cameras', im_h)

        return rgb_im, sem_im
    def print_image_to_folder(self, control):
        rgb_im, sem_im = self.grab_and_display_image()
        # grab the time
        time_grab = time.time_ns()
        # normalize controls to between 0 and 100, with 50 being straight or nothing
        norm_steer = (control.steer + 1) /2
        norm_throttle = control.throttle
        norm_brake = control.brake
        
        # TODO: make path part of the class
        cv2.imwrite('out_sem/rgb/%06d_%.3f_%.3f_%.3f.png' % (time_grab, norm_steer, norm_throttle, norm_brake), rgb_im)
        cv2.imwrite('out_sem/sem/%06d_%.3f_%.3f_%.3f.png' % (time_grab, norm_steer, norm_throttle, norm_brake), sem_im)
    
    def distance(self, location1, location2):
        return math.sqrt((location1.x - location2.x)**2 + (location1.y - location2.y)**2 + (location1.z - location2.z)**2)

    # function that advances vehicle by ONE tick. main loop is run by fleet manager
    def run_step(self):
        if self.vehicle is None or self.agent is None:
            return
        try:
            control = self.agent.run_step()
#             print(control)
            self.vehicle.apply_control(control)
            current_time = time.time()
            
            # light turns
            if abs(control.steer * 100) >= 5 and abs(control.steer * 100) < 10:
                if current_time - self.last_capture_time >= self.light_steer_interval:
                    self.print_image_to_folder(control)
                    self.last_capture_time = current_time
                    
            # sharp turn
            elif abs(control.steer * 100) >= 10:
                if current_time - self.last_capture_time >= self.sharp_steer_interval:
                    self.print_image_to_folder(control)
                    self.last_capture_time = current_time
            
            # drive straight
            elif current_time - self.last_capture_time >= self.interval: 
                self.print_image_to_folder(control)
                self.last_capture_time = current_time
                
            # check if vehicle has arrived at its destination    
            if self.agent.done():
                while True:
                    new_waypoint = random.choice(self.waypoint_list)
                    
                    if self.current_waypoint is None or self.distance(self.current_waypoint, new_waypoint.transform.location) > 50:  # Check if new waypoint is at least 50 meters away
                        break
                print("reached waypoint, heading to: ", new_waypoint) 
                self.agent.set_destination(new_waypoint.transform.location)
                self.current_waypoint = new_waypoint.transform.location
                
        except Exception as e:
            print("Caught RuntimeError while trying to control the vehicle:", e)

In [59]:
def on_collision(event, vehicle, respawn=False):
    
    print("Collision detected: Actor %s hit %s" % (event.other_actor.type_id, event.actor.type_id))
    # Stop the vehicle (or apply any other logic you deem necessary)
    vehicle.destroy()
    
# set to synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True  # Enables synchronous mode
settings.fixed_delta_seconds = 0.025  # Sets the time step for each tick
world.apply_settings(settings)
try:
    vehicle = Vehicle(world, 1, on_collision)
    vehicle2= Vehicle(world, 2, on_collision)

    vehicle.spawn()
    vehicle2.spawn(spawn_point=3)
#     world.tick()
    
#     vehicle.run_step()
    while True:
        vehicle.run_step()
        vehicle2.run_step()
        world.tick()
        
# #         if input("Press q to quit: ") == 'q':
# #             break
        if keyboard.is_pressed('q'):  # Check if 'q' is pressed
            print("Quitting simulation.")
            break
        if vehicle.vehicle.is_alive == False:
            break
    
finally:
    
    settings.synchronous_mode = False
    settings.fixed_delta_seconds = None
    world.apply_settings(settings)
    clear_world()
    cv2.destroyAllWindows()

spawning vehicle:  1
39.034509991537384
spawning vehicle:  2
45.058888986075935
VehicleControl(throttle=0.750000, steer=-0.100000, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=0.100000, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.200000, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=0.200000, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.300000, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=0.300000, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.400000, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, 

VehicleControl(throttle=0.750000, steer=0.004856, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.658655, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.007043, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.693993, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.007160, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.726847, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=0.001046, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.626847, brake=0.000000, hand

VehicleControl(throttle=0.000000, steer=-0.003581, brake=0.289702, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=0.178449, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.069465, steer=0.038230, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.387699, steer=0.132042, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.621657, steer=0.000437, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.000000, steer=0.032042, brake=0.264410, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.005146, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.000000, steer=-0.023468, brake=0.300000, hand_br

VehicleControl(throttle=0.750000, steer=-0.027463, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=0.001071, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.405978, steer=-0.021826, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=-0.020983, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.000000, steer=0.072215, brake=0.004991, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=0.001270, brake=0.000000, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.000000, steer=-0.015685, brake=0.001227, hand_brake=False, reverse=False, manual_gear_shift=False, gear=0)
VehicleControl(throttle=0.750000, steer=0.006155, brake=0.000000, hand_b

In [5]:

def print_vehicle_controls(vehicle):
    control = vehicle.get_control()
    normalized_steer = (control.steer + 1) /2
    normalized_throttle = (control.throttle)
    normalized_brake = (control.brake)
    print("Steer: {:.3f}, Throttle: {:.3f}, Brake: {:.3f}".format(normalized_steer, normalized_throttle, normalized_brake))




def on_collision(event, vehicle, respawn=False):
    
    print("Collision detected: Actor %s hit %s" % (event.other_actor.type_id, event.actor.type_id))
    # Stop the vehicle (or apply any other logic you deem necessary)
    vehicle.destroy()

def respawn_car(spawnPoint=None):
    global vehicle, agent, collision_sensor, camera_sem, camera_rgb, camera_data
    # Destroy existing actors if they exist
    
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
    if vehicle is not None:
        vehicle.destroy()
        vehicle = None
    if collision_sensor is not None:
        collision_sensor.destroy()
        collision_sensor = None
    if camera_sem is not None:
        camera_sem.destroy()
        camera_sem = None
    if camera_rgb is not None:
        camera_rgb.destroy()
        camera_rgb = None
    
    
    # if spawnpoint is unspecified, spawn new vehicle at random place
    
    if (spawnPoint):
        random_spawnpoint = spawnPoint
    
    print("Vehicle spawned at spawn point: ", random_spawnpoint)
    random_spawnpoint = random.randint(0, 10)  # Both 1 and 10 are included
    vehicle = spawn_vehicle(random_spawnpoint)
    
    # sometimes the vehicle isn't spawned correctly and 
    # other threads try to read it and then it gets fucked
    if vehicle is None:
        raise Exception("Failed to spawn vehicle")
        time.sleep(1)

    # delay for a second just in case
    time.sleep(1)
    
    # Setup collision sensor
    collision_sensor = setup_collision_sensor(vehicle, on_collision)

    # Setup cameras
    camera_sem, camera_rgb, camera_data = setup_cameras(vehicle)

    # Setup or reset the agent
    agent = setAgent(vehicle)

    print("New vehicle and sensors spawned. Continuing simulation...")

In [10]:
# helper functions for the main loop

def get_random_waypoint(world):
    map = world.get_map()
    # Generate waypoints every 2 meters
    waypoints = map.generate_waypoints(distance=2.0)
    return random.choice(waypoints)

def distance(location1, location2):
    return math.sqrt((location1.x - location2.x)**2 + (location1.y - location2.y)**2 + (location1.z - location2.z)**2)

# returns the openCV objects of rgb/segmented images from CARLA
def grab_pictures():
#     print_vehicle_controls(vehicle)
        
#     steering_wheel_direction = control.steer
    rgb_im = camera_data['rgb_image']
    sem_im = camera_data['sem_image']

    # display images
    im_h = cv2.hconcat([rgb_im,sem_im])
    cv2.imshow('2 cameras', im_h)
    
    return rgb_im, sem_im

In [9]:
# initialize global variables and create the car
    
vehicle = None
agent = None
collision_sensor = None
camera_sem = None
camera_rgb = None
camera_data = None
start_point = None

respawn_car()

10
New vehicle and sensors spawned. Continuing simulation...


In [ ]:
# function to capture CARLA's segmented images.
# as of now, ./out_sem/rgb and ./out_sem/sem folder for it to export properly.


# TODO: will refactor the code to make it object-oriented and implement thread locks
last_capture_time = time.time()
interval = 7.5 # set interval to 10 seconds
sharp_steer_interval = 0.1
light_steer_interval = 0.75


    
current_waypoint = None

if vehicle is None:
    respawn_car()
while True:
    
    # advance the world
    world.tick()

    if vehicle is not None and agent is not None:
        try:
            control = agent.run_step()

            vehicle.apply_control(control)
            current_time = time.time()
            control = vehicle.get_control()

            # note try to factor this out later lol
            # on light turn, delay 0.5
            if abs(control.steer * 100) >= 5 and abs(control.steer * 100) < 10:


                if current_time - last_capture_time >= light_steer_interval:
                    rgb_im, sem_im = grab_pictures()
                    # grab the time
                    time_grab = time.time_ns()
                    # normalize controls to between 0 and 100, with 50 being straight or nothing
                    normalized_steer = (control.steer + 1) /2
                    normalized_throttle = (control.throttle)
                    normalized_brake = (control.brake)
                    cv2.imwrite('out_sem/rgb/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), rgb_im)
                    cv2.imwrite('out_sem/sem/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), sem_im)
                    last_capture_time = current_time  # Update time

            # on sharp turn, delay 0.1
            elif abs(control.steer * 100) >= 10:
                if current_time - last_capture_time >= sharp_steer_interval:
                    # normalize controls to between 0 and 100, with 50 being straight or nothing
                    rgb_im, sem_im = grab_pictures()
                    # grab the time
                    time_grab = time.time_ns()
                    normalized_steer = (control.steer + 1) /2
                    normalized_throttle = (control.throttle)
                    normalized_brake = (control.brake)
                    cv2.imwrite('out_sem/rgb/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), rgb_im)
                    cv2.imwrite('out_sem/sem/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), sem_im)
                    last_capture_time = current_time  # Update time


            elif current_time - last_capture_time >= interval:        
                rgb_im, sem_im = grab_pictures()

                time_grab = time.time_ns()

                normalized_steer = (control.steer + 1) /2
                normalized_throttle = (control.throttle)
                normalized_brake = (control.brake)
                cv2.imwrite('out_sem/rgb/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), rgb_im)
                cv2.imwrite('out_sem/sem/%06d_%.3f_%.3f_%.3f.png' % (time_grab, normalized_steer, normalized_throttle, normalized_brake), sem_im)        
                # update time
                last_capture_time = current_time
            if cv2.waitKey(1) == ord('q'):
                break
            if agent.done():
                while True:
                    new_waypoint = get_random_waypoint(world)
                    if current_waypoint is None or distance(current_waypoint.transform.location, new_waypoint.transform.location) > 50:  # Check if new waypoint is at least 50 meters away
                        break
                print("reached waypoint, heading to: ", new_waypoint) 
                agent.set_destination(new_waypoint.transform.location)
                current_waypoint = new_waypoint
        except Exception as e:
            print("Caught RuntimeError while trying to control the vehicle:", e)
            time.sleep(1)
    

clear_world()
cv2.destroyAllWindows()

Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined
Caught RuntimeError while trying to control the vehicle: name 'control' is not defined


In [11]:
from predict_control import predict


# if vehicle is None:
respawn_car()
    
# see if you can keep a '3 second log' and then write to it when you crash
while True:
    
    world.tick()
    try:
        rgb_im, sem_im = grab_pictures()

        prediction = predict(sem_im, CARLA=True)

        new_steering = 2 * (prediction[0] - 0.5)
        print(f"steer: {new_steering}, throttle: {prediction[1]}, brake: {prediction[2]}")
        control = carla.VehicleControl(steer=new_steering, throttle=prediction[1], brake=0.0)
        vehicle.apply_control(control)
        current_time = time.time()
    except Exception as e:
        print(e)
        time.sleep(1)
        respawn_car()
#     control = vehicle.get_control()
    
    if cv2.waitKey(1) == ord('q'):
        break
clear_world()
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Clarence\\Documents\\Coding\\cs141\\researchProject\\selfDrivingCar\\Self_Driving_Car\\models\\driver_model-3.pt'

In [18]:
#grab a snapshot from the camera an show in a pop-up window
img = camera_data['image']
cv2.imshow('RGB Camera',img)
cv2.waitKey(0)

113

In [82]:
# clean up after yourself

# camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [7]:
len(spawn_points)

10422

In [8]:
start_point = spawn_points[0]
spectator = world.get_spectator()
spectator.set_transform(start_point)

In [12]:
spectator.set_transform(carla.Transform(carla.Location(x=-1085.286377, y=3112.225830, z=356.060608), carla.Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000)))

In [11]:
print(start_point)

Transform(Location(x=-1085.286377, y=3112.225830, z=356.060608), Rotation(pitch=1.648070, yaw=20.234367, roll=0.000000))
